# Generative Adversarial Networks
### Example 1

Let’s walk through a simplified example of a Generative Adversarial Network (GAN), which consists of two neural networks: a Generator and a Discriminator. The generator tries to create fake data that looks real, while the discriminator tries to distinguish between real and fake data. Over time, the generator gets better at creating realistic data, and the discriminator improves at detecting it.

Here's an overview of the steps we’ll take:
- Define the Generator and Discriminator Networks.
- Set Up the Training Loop where both networks train in tandem.
- Use Random Noise to Generate Fake Data through the generator.
- Train the Discriminator to classify real vs. fake data.
- Update the Generator to produce more convincing fake data based on the discriminator's feedback.

Below is a simple GAN example using PyTorch. This example will train a GAN on random 1-dimensional data, which can be extended to images or other types of data.

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Step 1: Define Generator and Discriminator Networks

class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim),
            nn.Tanh()  # Outputs values in the range [-1, 1]
        )

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()  # Outputs probability [0, 1], real (1) or fake (0).
        )

    def forward(self, x):
        return self.model(x)

# Step 2: Initialize Networks, Optimizers, and Loss Function
latent_dim = 10  # Size of the noise vector input to the Generator
data_dim = 1     # Output dimension of data
generator = Generator(latent_dim, data_dim)
discriminator = Discriminator(data_dim)

# Use Binary Cross-Entropy loss and separate optimizers for each network
# Creates a criterion that measures the Binary Cross Entropy between the target and the input probabilities
criterion = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=0.0002, maximize=False)
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002, maximize=False)

# Step 3: Training Loop
num_epochs = 5000
for epoch in range(num_epochs):
    # Train Discriminator
    # -------------------

    # Generate real data (random numbers centered around 0.5, ranging from 0.25 to 0.75)
    real_data = torch.rand(64, data_dim) * 0.5 + 0.25
    real_labels = torch.ones(64, 1)

    # Generate fake data using the generator
    noise = torch.randn(64, latent_dim) # 10 series of noise
    fake_data = generator(noise)
    fake_labels = torch.zeros(64, 1)

    # Calculate discriminator loss on real and fake data
    real_output = discriminator(real_data)
    fake_output = discriminator(fake_data.detach())  # detach to avoid training the generator here
    loss_real = criterion(real_output, real_labels)
    loss_fake = criterion(fake_output, fake_labels)
    loss_d = (loss_real + loss_fake)/2

    # Backpropagate and optimize discriminator
    optimizer_d.zero_grad()
    loss_d.backward()
    optimizer_d.step()

    # Train Generator
    # ----------------

    # Generate fake data again for updating the generator
    noise = torch.randn(64, latent_dim)
    fake_data = generator(noise)
    fake_output = discriminator(fake_data)
    loss_g = criterion(fake_output, real_labels)  # Aim for the generator to produce data that fools the discriminator

    # Backpropagate and optimize generator
    optimizer_g.zero_grad()
    loss_g.backward()
    optimizer_g.step()

    # Print Losses
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss D: {loss_d.item():.4f}, Loss G: {loss_g.item():.4f}")

print("Training complete!")

Epoch 0, Loss D: 0.7153, Loss G: 0.5921
Epoch 500, Loss D: 0.6808, Loss G: 0.6407
Epoch 1000, Loss D: 0.6957, Loss G: 0.6884
Epoch 1500, Loss D: 0.6938, Loss G: 0.6927
Epoch 2000, Loss D: 0.6883, Loss G: 0.7358
Epoch 2500, Loss D: 0.6970, Loss G: 0.6516
Epoch 3000, Loss D: 0.6953, Loss G: 0.7007
Epoch 3500, Loss D: 0.7063, Loss G: 0.6558
Epoch 4000, Loss D: 0.6985, Loss G: 0.7090
Epoch 4500, Loss D: 0.6924, Loss G: 0.7103
Training complete!


**Explanation of the Code** <br>
- Generator: It takes a random noise vector (latent space) as input and outputs a data sample.
- Discriminator: It takes a data sample and outputs a probability indicating whether the sample is real (1) or fake (0).
- Training Loop:
    - Discriminator Training: We train the discriminator on both real data (labelled as 1) and fake data generated by the generator (labelled as 0). This helps the discriminator to learn to distinguish between real and fake samples.
    - Generator Training: We train the generator to produce samples that the discriminator misclassifies as real, effectively "fooling" the discriminator.

**Notes**
- Epochs: More epochs might be required to achieve realistic data.
- Tuning: Adjusting the architecture, learning rate, and other hyperparameters is key in practical GANs.

This example demonstrates the basic setup of a GAN for generating 1D data and can be expanded for more complex data, such as images.